In [21]:
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
import subprocess

In [4]:
#load data
X_test = np.load("C:/Users/sanda/Documents/esp_dev_files/data/ecg_specto_data/X_test.npy")
y_test = np.load("C:/Users/sanda/Documents/esp_dev_files/data/ecg_specto_data/y_test.npy")

In [16]:
print(X_test.shape)

(32, 570, 33)


In [8]:
model = tf.keras.models.load_model("C:/Users/sanda/Documents/esp_dev_files/tensor_4_ml/src/models/keras_ecg_cnn_small_9.h5")

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 570, 33, 32)       320       
                                                                 
 batch_normalization (BatchN  (None, 570, 33, 32)      128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 570, 33, 32)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 285, 17, 64)       18496     
                                                                 
 batch_normalization_1 (Batc  (None, 285, 17, 64)      256       
 hNormalization)                                                 
                                                                 
 activation_1 (Activation)   (None, 285, 17, 64)       0

In [15]:
print(model.inputs[0].shape.as_list())

[None, 570, 33, 1]


In [28]:
# input_ids = tf.keras.layers.Input((MAXLEN,), dtype=tf.int32, name='input_ids') 
# attention_mask = tf.keras.layers.Input((MAXLEN,), dtype=tf.int32, name='attention_mask') 
# inputs = [input_ids, attention_mask] output = tf_model(inputs[0], inputs[1]) 
# tf_model = tf.keras.Model(inputs=inputs, outputs=output) 
# tf_outputs = tf_model(inputs=[tf_inputs["input_ids"], tf_inputs[ "attention_mask"]]) 
batch_size = 1
input_shape = model.inputs[0].shape.as_list()
input_shape[0] = batch_size
func = tf.function(model).get_concrete_function(
    tf.TensorSpec(input_shape, model.inputs[0].dtype))
converter = tf.lite.TFLiteConverter.from_concrete_functions([func])
# converter = tf.lite.TFLiteConverter.from_keras_model(model) 
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS] 
tflite_model = converter.convert()

In [27]:


with open('C:/Users/sanda/Documents/esp_dev_files/tensor_4_ml/src/models/keras_ecg_cnn_small_9.tflite', 'wb+') as f:
    f.write(tflite_model)

# list_files = subprocess.run(["xxd", "-i", "C:/Users/sanda/Documents/esp_dev_files/tensor_4_ml/src/models/keras_ecg_cnn_small_9.tflite", ">", "C:/Users/sanda/Documents/esp_dev_files/tensor_4_ml/src/models/cnn_small_9.cc"])